**Поиск токсичных комментариев**

**Постановка задачи**
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**Содержание**

1. Подготовка данных
- Загрузка библиотек и датасета
- Формирование функции для лемматизации
- Создание корпусов признаков и целевого признака
- Разделение датасета на обучающую и тестовую выборки
2. Обучение
- Модель решающего дерева
- Модель логистической регрессии
- CatBoost Classifier
3. Выводы по проекту

# Подготовка данных

## Загрузка библиотек и датасета

In [1]:
# Для начала подгрузим необходимые для дальнейшей работы библиотеки и инструменты

import pandas as pd
import numpy as np

import torch
import transformers
from tqdm import notebook
from tqdm import tqdm

from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer

# модели
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

# метрика
from sklearn.metrics import f1_score

from datetime import datetime, time

import warnings
warnings.simplefilter("ignore")

start_full_time = datetime.now() 

In [2]:
# Загружаем файл с исходным датасетом и выводим его первые строки для визуального ознакомления

try:
    df = pd.read_csv('toxic_comments.csv')
    df.head(10)
except:
    df = pd.read_csv('/datasets/toxic_comments.csv')
    df.head(10)

In [3]:
# Для оценки состава датасета вызовем метод info()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


**Итого**, наш датасет состоит из 159571 объектов (строк) и всего 2 признаков:
    
- text - текст комментария, пропусков нет;
- toxic - бинарный признак, где 1 - обозначение токсичного комментария, а 0 - нетоксичного, пропусков нет.

## Формирование функции для лемматизации

In [4]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\konst\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\konst\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# Создадим класс лемматизатора

lemmatizer = WordNetLemmatizer()

In [6]:
# Для визуализации прогресса выполнения функции 
# применим метод tqdm()

tqdm.pandas()

# создадим функцию для лемматизации и очистки текста

def clear_text(text):
    word_list = nltk.word_tokenize(text) 
    clear_list = re.sub(r'[^A-Za-z]', ' ', ' '.join(word_list))
    lemm_text = ' '.join([lemmatizer.lemmatize(w) for w in word_list]) 

    return lemm_text

In [7]:
# Применим функцию к набору данных

df['lemm_text'] = df['text'].progress_apply(lambda x: clear_text(x))

100%|████████████████████████████████████████████████████████████████████████| 159571/159571 [01:38<00:00, 1615.90it/s]


In [8]:
df

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww ! He match this background colour I 'm s...
2,"Hey man, I'm really not trying to edit war. It...",0,"Hey man , I 'm really not trying to edit war ...."
3,"""\nMore\nI can't make any real suggestions on ...",0,`` More I ca n't make any real suggestion on i...
4,"You, sir, are my hero. Any chance you remember...",0,"You , sir , are my hero . Any chance you remem..."
...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,`` : : : : : And for the second time of asking...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That is a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,"Spitzer Umm , there no actual article for pros..."
159569,And it looks like it was actually you who put ...,0,And it look like it wa actually you who put on...


## Создание корпусов признаков и целевого признака

In [9]:
# В отдельные переменные записываем признаки из датасета, а также целевой признак

corpus = df['lemm_text']
target = df['toxic']

In [10]:
# Обучающие признаки переводим в формат юникода

corpus = corpus.values.astype('U')

## Разделение датасета на обучающую и тестовую выборки

In [11]:
# Отделим от датасета тренировочнуи выборку из 80 % данных

df_train, df_last = train_test_split(df, test_size=0.2, random_state=12345)

In [12]:
# Делим "отрезанные" 20 % на тестовую валидационную выборки в соотношении 1:9

df_valid, df_test = train_test_split(df_last, test_size=0.5, random_state=12345)

In [13]:
# Для оценки сбалансированности датасета вызовем метод value_counts() для целевого признака

df_train['toxic'].value_counts()

0    114670
1     12986
Name: toxic, dtype: int64

Отмечается значительный дисбаланс классов с перекосом в сторону нулей. Чтобы это исправить, необходимо применить upsampling

In [14]:
# Разберем объекты по классам и запишем их в отдельные переменные

df_nulls = df_train[df_train['toxic'] == 0]
df_ones = df_train[df_train['toxic'] == 1]

In [15]:
# Применим метод resample

df_nulls_downsampled  = resample(df_nulls, replace=True, n_samples=df_train['toxic'].value_counts()[1]*2, random_state=12345)

In [16]:
# Объединим полученные выборки

df_train = pd.concat([df_nulls_downsampled, df_ones])

In [17]:
# Проверяем результат работы

df_train['toxic'].value_counts()

0    25972
1    12986
Name: toxic, dtype: int64

In [18]:
# Отделяем обучающие признаки от целевого

corpus_train = df_train['lemm_text']
corpus_valid = df_valid['lemm_text']
corpus_test = df_test['lemm_text']

target_train = df_train['toxic']
target_valid = df_valid['toxic']
target_test = df_test['toxic']

In [19]:
# В целях формирования новых признаков для моделей рассчитаем и запишем в датасет TF-IDF,
# при этом определим стоп-слова для их отбрасывания

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(min_df=2, stop_words=stopwords)
count_tf_idf.fit(corpus_train)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\konst\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TfidfVectorizer(min_df=2,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

In [20]:
# Формируем выборки обучающих, валидационных и тестовых признаков

features_train = count_tf_idf.transform(corpus_train).toarray()
features_valid = count_tf_idf.transform(corpus_valid).toarray()
features_test = count_tf_idf.transform(corpus_test).toarray()

features_train.shape, features_valid.shape, features_test.shape

((38958, 31980), (15957, 31980), (15958, 31980))

# Обучение

## Модель решающего дерева

In [21]:
# Подбираем оптимальные гиперпараметры модели с помощью цикла

best_model_tree = None
best_result_tree = 0
best_depth_tree = 0

for depth in range(1, 3):
    model_tree = DecisionTreeClassifier(max_depth = depth,  random_state = 12345)
    model_tree.fit(features_train, target_train)
    prediction_tree_valid = model_tree.predict(features_valid)
    f1_tree = f1_score(target_valid, prediction_tree_valid)
    
    if f1_tree > best_result_tree:
        best_model_tree = model_tree
        best_result_tree = f1_tree
        best_depth_tree = depth

In [22]:
best_result_tree

0.3714710252600297

In [23]:
%%time
start_time = datetime.now() 

# Обучаем модель на подобранных гиперпараметрах

model_tree = DecisionTreeClassifier(max_depth = best_depth_tree,  random_state = 12345)
model_tree.fit(features_train, target_train)
        
time_tree = datetime.now() - start_time
print(str(time_tree))

0:01:27.904777
Wall time: 1min 27s


In [24]:
# Получим предсказания на тестовой выборке и рассчитаем метрику F1

preds_test = best_model_tree.predict(features_test)
f1_tree_test = f1_score(target_test, preds_test)
f1_tree_test

0.3680241327300151

## Модель логистической регрессии

In [25]:
%%time
start_time = datetime.now() 

# Обучим модель логистической регрессии


model_lr = (LogisticRegression(solver='liblinear', 
                               random_state=12345)
            .fit(features_train, target_train))
            
time_lr = datetime.now() - start_time
print(str(time_lr))

0:00:16.860794
Wall time: 16.9 s


In [26]:
# Получаем предсказания на тренировочной выборке и рассчитаем метрику F1

predictions = model_lr.predict(features_valid)
f1_valid_lr = f1_score(target_valid, predictions)
f1_valid_lr

0.7484370348317951

In [27]:
# Получаем предсказания на тестовой выборке и рассчитаем метрику F1

predictions_test = model_lr.predict(features_test)
f1_test_lr = f1_score(target_test, predictions_test)
f1_test_lr

0.7599510104102878

## CatBoost Classifier

In [28]:
%%time
start_time = datetime.now() 

# Обучим модель CatBoost Classifier

model_cat = CatBoostClassifier(loss_function= 'Logloss', 
                               iterations=300,
                               eval_metric='F1',
                               random_state=12345)

model_cat.fit(features_train, target_train, verbose=25)
prediction = model_cat.predict(features_test)
            
time_cat = datetime.now() - start_time
print(str(time_cat))

Learning rate set to 0.148456
0:	learn: 0.4723818	total: 357ms	remaining: 1m 46s
25:	learn: 0.7148134	total: 4.66s	remaining: 49.1s
50:	learn: 0.7660957	total: 8.97s	remaining: 43.8s
75:	learn: 0.7915156	total: 13.3s	remaining: 39.1s
100:	learn: 0.8149594	total: 17.6s	remaining: 34.7s
125:	learn: 0.8283847	total: 21.8s	remaining: 30.2s
150:	learn: 0.8396529	total: 26.1s	remaining: 25.7s
175:	learn: 0.8476096	total: 30.3s	remaining: 21.4s
200:	learn: 0.8543981	total: 34.6s	remaining: 17s
225:	learn: 0.8625346	total: 38.8s	remaining: 12.7s
250:	learn: 0.8677069	total: 43s	remaining: 8.4s
275:	learn: 0.8723386	total: 47.3s	remaining: 4.11s
299:	learn: 0.8751739	total: 51.3s	remaining: 0us
0:07:36.743058
Wall time: 7min 36s


In [29]:
# Получаем  метрику F1 на тренировочных данных

f1_train_cat = model_cat.get_best_score()['learn']['F1']
f1_train_cat

0.8751738965473632

In [30]:
# Получаем предсказания на тестовой выборке и рассчитаем метрику F1

f1_test_cat = f1_score(target_test, prediction)
f1_test_cat

0.7586637527318139

## Итоговая таблица

In [31]:
# Сгруппируем результаты в отдельную таблиуцу

table_final = pd.DataFrame({'Модель': ['LogisticRegression', 'DecisionTreeClassifier', 'CatBoostClassifier'],
                            'F1_valid': [f1_valid_lr, best_result_tree, f1_train_cat],
                            'F1_test': [f1_test_lr, f1_tree_test, f1_test_cat],
                            'Скорость обучения': [time_lr, time_tree, time_cat]})

table_final

,Модель,F1_valid,F1_test,Скорость обучения
0,LogisticRegression,0.748437,0.759951,0 days 00:00:16.860794
1,DecisionTreeClassifier,0.371471,0.368024,0 days 00:01:27.904777
2,CatBoostClassifier,0.875174,0.758664,0 days 00:07:36.743058


In [32]:
full_time = datetime.now() - start_full_time
full_time

datetime.timedelta(seconds=826, microseconds=32886)

# Выводы по проекту

В данной работе были отточены навыки загрузки необходимых библиотек, подготовки данных к работе, балансировки классов, лемматизации текстов, создания корпусов, разбития датасета на выборки.
    
Затем были обучены  ML-модели в версиях классификаторов - Логистическая Регрессия, Решающее Древо, CatBoost. Для каждой из них были получены предсказания для тестовой выборки. Полученные результаты позволяют судить о следующем:
- модель решающего древа оказалась наименее пригодной для классификации текстов - обучается долго, налицо переобучение, метрика мала;
- логистическая регрессия, несмотря на ее простоту, показала самую высокую метрику и быстрое обучение. 
- CatBoost демонстрирует самое высокое качество, но время обучения значительно больше, чем у LR.
   
Таким образом, логистическая регрессия рекомендована для использования в разрабатываемом продукте.